In [0]:
!wget -q https://www.dropbox.com/s/wr1fjnnt254rcq7/Patrick%20Jean-Baptiste%20-%20Emotion_Images.zip?dl=1 -O Emotion_Images.zip

!wget -q https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Song_Actors_01-24.zip
!wget -q https://zenodo.org/api/files/c8f9b6fe-82ac-481c-ad9c-12b5581cb4b4/Audio_Speech_Actors_01-24.zip

!unzip -q -d ./song Audio_Song_Actors_01-24.zip 
!unzip -q -d ./speech Audio_Speech_Actors_01-24.zip
!unzip -q Emotion_Images.zip

!wget -q https://www.dropbox.com/s/yigijs122togfk4/embeddings.dat?dl=1  -O embeddings.dat
!wget -q https://www.dropbox.com/s/qdhtexle4p0ngc3/DatasetSplitCSV.zip?dl=1 -O DatasetSplitCSV.zip
!unzip -q DatasetSplitCSV.zip


In [0]:
!wget -q https://www.dropbox.com/s/yigijs122togfk4/embeddings.dat?dl=1  -O embeddings.dat
!wget -q https://www.dropbox.com/s/qdhtexle4p0ngc3/DatasetSplitCSV.zip?dl=1 -O DatasetSplitCSV.zip
!unzip -q DatasetSplitCSV.zip

In [0]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import glob

In [0]:
def mapReduce(embed,funcName):
    if funcName=="Avg":
        embed= [np.average(embed,axis=0)]
    if funcName=="Pad":
        embed=np.pad(embed, [( 0,6-embed.shape[0]), (0, 0)], mode='constant', constant_values=0)
        embed= [embed.reshape(-1)]
    if funcName=="Many2One":
        embed=[embed[i,:] for i in range(embed.shape[0])]
    if funcName=="None":
        embed=[embed]
    return embed

def uint8_to_float32(x):
    return (np.float32(x) - 128.) / 128.

In [0]:
#Parameters
params={}
params["embeddingType"]="post" # post,raw
params["mapReduceFunc"]="Many2One" # Avg,Pad,Many2One

In [0]:
train=pd.read_csv("train.csv",header=None)
valid=pd.read_csv("valid.csv",header=None)
test=pd.read_csv("test.csv",header=None)

embeddings=np.load("embeddings.dat",allow_pickle=True)
# embeddings["post"]['03-02-04-01-02-02-02.wav'].dtype
seconds={}

for key,embed in embeddings["post"].items():
    embeddings["post"][key]=uint8_to_float32(embed)
    seconds[key]=embed.shape[0]


In [0]:
x_train=[]
x_val=[]
x_test=[]
for x in train[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_train.extend(mapReduce(embed,params["mapReduceFunc"]))

for x in valid[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_val.extend(mapReduce(embed,params["mapReduceFunc"]))

for x in test[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_test.extend(mapReduce(embed,params["mapReduceFunc"]))

x_train=np.array(x_train)
x_val=np.array(x_val)
x_test=np.array(x_test)

y_train=np.array(train[1].astype('category').cat.codes)
y_val=np.array(valid[1].astype('category').cat.codes)
y_test=np.array(test[1].astype('category').cat.codes)

if params["mapReduceFunc"]=="Many2One":
    # print("a")
    y_train2,y_val2,y_test2=[],[],[]
    for x,y,y2 in zip([train[0],valid[0],test[0]],[y_train,y_val,y_test],[y_train2,y_val2,y_test2]):
        for i,embedName in enumerate(x):
            # print(seconds[embedName])
            y2.extend([y[i] for k in range(seconds[embedName])])
        # print(y2)
    y_train,y_val,y_test=[np.array(y) for y in [y_train2,y_val2,y_test2]]


In [0]:
imageList=glob.glob("Emotion_Images/*")
imageDict={}
for img in imageList:
    fname=(img).split("/")[-1].split(".")[0]+".wav"
    imageDict[fname]=img
    

In [0]:
x_trainImg=[]
x_valImg=[]
x_testImg=[]
for x in train[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_trainImg.extend(mapReduce(embed,params["mapReduceFunc"]))

for x in valid[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_valImg.extend(mapReduce(embed,params["mapReduceFunc"]))

for x in test[0]:
    embed=embeddings[params["embeddingType"]][x]
    x_testImg.extend(mapReduce(embed,params["mapReduceFunc"]))

x_trainImg=np.array(x_trainImg)
x_valImg=np.array(x_valImg)
x_testImg=np.array(x_testImg)